In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,mutual_info_regression

In [2]:
data= pd.read_csv('../Data/Chennai houseing sale.csv')

In [3]:
data.head()

,PRT_ID,AREA,INT_SQFT,DATE_SALE,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,SALE_COND,PARK_FACIL,...,UTILITY_AVAIL,STREET,MZZONE,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_OVERALL,REG_FEE,COMMIS,SALES_PRICE
0,P03210,Karapakkam,1004,04-05-2011,131,1.0,1.0,3,AbNormal,Yes,...,AllPub,Paved,A,4.0,3.9,4.9,4.330,380000,144400,7600000
1,P09411,Anna Nagar,1986,19-12-2006,26,2.0,1.0,5,AbNormal,No,...,AllPub,Gravel,RH,4.9,4.2,2.5,3.765,760122,304049,21717770
2,P01812,Adyar,909,04-02-2012,70,1.0,1.0,3,AbNormal,Yes,...,ELO,Gravel,RL,4.1,3.8,2.2,3.090,421094,92114,13159200
3,P05346,Velachery,1855,13-03-2010,14,3.0,2.0,5,Family,No,...,NoSewr,Paved,I,4.7,3.9,3.6,4.010,356321,77042,9630290
4,P06210,Karapakkam,1226,05-10-2009,84,1.0,1.0,3,AbNormal,Yes,...,AllPub,Gravel,C,3.0,2.5,4.1,3.290,237000,74063,7406250


In [4]:
data['TOTAL_PRICE']=data['REG_FEE']+data['COMMIS']+data['SALES_PRICE']

In [5]:
data.drop(['REG_FEE','COMMIS','PRT_ID','SALES_PRICE'],axis=1, inplace=True)

In [6]:
data.head()

,AREA,INT_SQFT,DATE_SALE,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,SALE_COND,PARK_FACIL,DATE_BUILD,BUILDTYPE,UTILITY_AVAIL,STREET,MZZONE,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_OVERALL,TOTAL_PRICE
0,Karapakkam,1004,04-05-2011,131,1.0,1.0,3,AbNormal,Yes,15-05-1967,Commercial,AllPub,Paved,A,4.0,3.9,4.9,4.330,8124400
1,Anna Nagar,1986,19-12-2006,26,2.0,1.0,5,AbNormal,No,22-12-1995,Commercial,AllPub,Gravel,RH,4.9,4.2,2.5,3.765,22781941
2,Adyar,909,04-02-2012,70,1.0,1.0,3,AbNormal,Yes,09-02-1992,Commercial,ELO,Gravel,RL,4.1,3.8,2.2,3.090,13672408
3,Velachery,1855,13-03-2010,14,3.0,2.0,5,Family,No,18-03-1988,Others,NoSewr,Paved,I,4.7,3.9,3.6,4.010,10063653
4,Karapakkam,1226,05-10-2009,84,1.0,1.0,3,AbNormal,Yes,13-10-1979,Others,AllPub,Gravel,C,3.0,2.5,4.1,3.290,7717313


In [7]:
x_train, x_test, y_train, y_test = train_test_split(data.loc[:,:'QS_OVERALL'], data['TOTAL_PRICE'],test_size=0.15)

In [8]:
x_train.shape, x_test.shape

((6042, 18), (1067, 18))

## -- Fixing redundancy of values in object datatype features --

In [9]:
# Getting all features with object datatype in obj_features variable
obj_features=[features for features in data.columns if data[features].dtypes=='object' and 'DATE' not in features]

In [10]:
val_dict={'AREA':{'Karapakkam':['Karapakam'], 'Anna Nagar':['Ana Nagar', 'Ann Nagar'], 'Adyar':['Adyr'], 'Velachery':['Velchery'], 
 'Chrompet':['Chrompt', 'Chrmpet', 'Chormpet'], 'KK Nagar':['KKNagar'],'T Nagar': ['TNagar']},
'SALE_COND':{'AbNormal':['Ab Normal'], 'Partial':['Partiall', 'PartiaLl'], 'AdjLand':['Adj Land']},
'PARK_FACIL': {'No':['Noo']},
'BUILDTYPE':{'Commercial':['Comercial'], 'Others': ['Other']},
'UTILITY_AVAIL': {'AllPub':['All Pub'], 'NoSewa': ['NoSewr ','NoSwer','NoSeWa'], },
'STREET': {'Paved':['Pavd'], 'No Access':['NoAccess']}

}

In [11]:
# Replacing all misspelled values with original values
def reduce_redundant_val(data,val_dict):
    for feature in val_dict:
        for classes in val_dict[feature]:
            for val in val_dict[feature][classes]:
                data[feature]=data[feature].replace(val,classes)

In [12]:
reduce_redundant_val(x_train,val_dict)
reduce_redundant_val(x_test,val_dict)

## -- Performing Label Encoding over Object Datatype Features --

In [13]:
label_encoder=LabelEncoder()
for feature in obj_features:
    label_encoder.fit(x_train[feature])
    x_train[feature+'_enc']=label_encoder.transform(x_train[feature])
    x_test[feature+'_enc']=label_encoder.transform(x_test[feature])

In [14]:
x_train.drop(obj_features,axis=1,inplace=True)
x_test.drop(obj_features,axis=1,inplace=True)

In [15]:
x_train.head()

,INT_SQFT,DATE_SALE,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,DATE_BUILD,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_OVERALL,AREA_enc,SALE_COND_enc,PARK_FACIL_enc,BUILDTYPE_enc,UTILITY_AVAIL_enc,STREET_enc,MZZONE_enc
6788,1726,02-12-2008,112,2.0,1.0,4,09-12-1980,4.8,3.8,5.0,4.580,3,1,0,0,2,2,5
2690,1921,26-11-2012,146,2.0,1.0,5,01-12-1993,2.0,2.1,3.4,2.585,1,2,1,0,2,1,4
416,702,07-03-2012,190,1.0,1.0,2,12-03-1994,2.0,2.6,3.0,2.540,2,1,1,1,1,1,5
6774,2178,13-08-2009,188,3.0,2.0,5,16-08-1999,3.3,4.5,2.3,3.260,3,3,0,0,2,0,3
1212,790,25-02-2012,119,1.0,1.0,2,01-03-1994,4.9,2.6,2.0,3.050,4,1,0,0,0,2,0


## -- Converting Temporal data to Numeric feature --

In [16]:
# Converting DATE_SALE and DATE_BUILD feature to House_age feature

x_train['DATE_BUILD']=pd.to_datetime(x_train['DATE_BUILD'],format='%d-%M-%Y')
x_train['DATE_SALE']=pd.to_datetime(x_train['DATE_SALE'],format='%d-%M-%Y')

x_train['HOUSE_AGE']=x_train['DATE_SALE'].dt.year-x_train['DATE_BUILD'].dt.year

x_train.drop(['DATE_BUILD', 'DATE_SALE'],axis=1,inplace=True)

In [17]:
x_train.head()

,INT_SQFT,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_OVERALL,AREA_enc,SALE_COND_enc,PARK_FACIL_enc,BUILDTYPE_enc,UTILITY_AVAIL_enc,STREET_enc,MZZONE_enc,HOUSE_AGE
6788,1726,112,2.0,1.0,4,4.8,3.8,5.0,4.580,3,1,0,0,2,2,5,28
2690,1921,146,2.0,1.0,5,2.0,2.1,3.4,2.585,1,2,1,0,2,1,4,19
416,702,190,1.0,1.0,2,2.0,2.6,3.0,2.540,2,1,1,1,1,1,5,18
6774,2178,188,3.0,2.0,5,3.3,4.5,2.3,3.260,3,3,0,0,2,0,3,10
1212,790,119,1.0,1.0,2,4.9,2.6,2.0,3.050,4,1,0,0,0,2,0,18


In [19]:
x_test['DATE_BUILD']=pd.to_datetime(x_test['DATE_BUILD'],format='%d-%M-%Y')
x_test['DATE_SALE']=pd.to_datetime(x_test['DATE_SALE'],format='%d-%M-%Y')

x_test['HOUSE_AGE']=x_test['DATE_SALE'].dt.year-x_test['DATE_BUILD'].dt.year

x_test.drop(['DATE_BUILD', 'DATE_SALE'],axis=1,inplace=True)

In [20]:
x_test.head()

,INT_SQFT,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_OVERALL,AREA_enc,SALE_COND_enc,PARK_FACIL_enc,BUILDTYPE_enc,UTILITY_AVAIL_enc,STREET_enc,MZZONE_enc,HOUSE_AGE
572,1976,82,2.0,1.0,5,2.5,2.4,2.8,2.595,1,0,1,0,2,0,4,14
6420,1366,168,2.0,2.0,4,2.7,3.7,4.7,3.800,4,2,1,1,2,1,3,11
5021,1171,51,1.0,1.0,3,4.5,2.5,3.0,3.350,0,2,0,2,0,1,3,5
2328,1798,44,2.0,1.0,4,2.1,3.7,2.7,3.000,6,2,1,1,2,1,4,48
5224,844,101,1.0,1.0,2,4.0,2.9,4.7,3.950,4,2,0,0,2,0,0,12


## -- Imputing Missing values in training and test data --

In [21]:
imputer=KNNImputer()

In [22]:
imputer.fit(x_train)

KNNImputer()

In [23]:
x_train=pd.DataFrame(imputer.transform(x_train),columns=x_train.columns)
x_test=pd.DataFrame(imputer.transform(x_test),columns=x_test.columns)

In [24]:
x_test.head()

,INT_SQFT,DIST_MAINROAD,N_BEDROOM,N_BATHROOM,N_ROOM,QS_ROOMS,QS_BATHROOM,QS_BEDROOM,QS_OVERALL,AREA_enc,SALE_COND_enc,PARK_FACIL_enc,BUILDTYPE_enc,UTILITY_AVAIL_enc,STREET_enc,MZZONE_enc,HOUSE_AGE
0,1976.0,82.0,2.0,1.0,5.0,2.5,2.4,2.8,2.595,1.0,0.0,1.0,0.0,2.0,0.0,4.0,14.0
1,1366.0,168.0,2.0,2.0,4.0,2.7,3.7,4.7,3.800,4.0,2.0,1.0,1.0,2.0,1.0,3.0,11.0
2,1171.0,51.0,1.0,1.0,3.0,4.5,2.5,3.0,3.350,0.0,2.0,0.0,2.0,0.0,1.0,3.0,5.0
3,1798.0,44.0,2.0,1.0,4.0,2.1,3.7,2.7,3.000,6.0,2.0,1.0,1.0,2.0,1.0,4.0,48.0
4,844.0,101.0,1.0,1.0,2.0,4.0,2.9,4.7,3.950,4.0,2.0,0.0,0.0,2.0,0.0,0.0,12.0


## -- Feature Scaling --

In [25]:
scale=StandardScaler()

In [26]:
scale.fit(x_train)

StandardScaler()

In [27]:
scale_x_train=pd.DataFrame(scale.transform(x_train),columns=x_train.columns)
scale_x_test=pd.DataFrame(scale.transform(x_test),columns=x_test.columns)

## -- Feature Selection --

In [28]:
k_features=SelectKBest(mutual_info_regression,k=8)

In [29]:
k_features.fit(scale_x_train,y_train)

SelectKBest(k=8,
            score_func=<function mutual_info_regression at 0x000001B23CF93100>)

In [30]:
k_features.transform(scale_x_train)
scale_x_train=scale_x_train[k_features.get_feature_names_out()]
scale_x_test=scale_x_test[k_features.get_feature_names_out()]

In [31]:
scale_x_train.to_csv('../Data/processed_data/x_train.csv',index=False)
scale_x_test.to_csv('../Data/processed_data/x_test.csv',index=False)
y_train.to_csv('../Data/processed_data/y_train.csv',index=False)
y_test.to_csv('../Data/processed_data/y_test.csv',index=False)